In [10]:
import pandas as pd
import numpy as np

import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras import optimizers
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten, Dropout, BatchNormalization, Reshape, LeakyReLU, Input
from tensorflow.keras.models import Model

#  Generater

In [ ]:
def generator(data, lookback, min_index, max_index, batch_size,
             shuffle=False, step=1, delay=0):
    
    if max_index is None:
        max_index = len(data) - delay -1
        
    i = min_index + lookback
    
    while 1:
        if shuffle:
            rows = np.random.randint(min_index+lookback, max_index, size=batch_size)
    # min_index+lookback과 max_index 사이의 숫자를 batch_size만큼 만든다.
    # 단순히 batch를 만들어주기 위해 index를 rows에 저장  
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            
            rows = np.arrange(i, min(i+batch_size, max_index))
            i += len(rows)
        
        samples = np.zeros((len(rows), lookback//step , data.shape[-1]))
        targets = np.zeros((len(rows), lookback//step , data.shape[-1]))
        # shape = (batchsize, lookback, feature 수)
        
        for j, row in enumerate(rows):# j 는 0~batchsize, row는 index number
            indices = range(rows[j]-lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[indices]
        
        yield samples, targets

# 그냥 Autoencoder

## Encoder

In [11]:
encoder = Sequential([
    Input(shape=(28, 28, 1)),

    Conv2D(32, 3, padding='same'), 
    BatchNormalization(),
    LeakyReLU(),

    Conv2D(64, 3, strides=2, padding='same'),
    BatchNormalization(),
    LeakyReLU(),

    Conv2D(64, 3, strides=2, padding='same'),
    BatchNormalization(),
    LeakyReLU(),

    Conv2D(64, 3, padding='same'),
    BatchNormalization(),
    LeakyReLU(),

    Flatten(),
    Dense(16)
])

encoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 32)        128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)         

## Decoder

In [12]:

decoder = Sequential([
    Input(shape=(16)),

    Dense(7*7*64), 
    Reshape((7, 7, 64)),
    
    Conv2DTranspose(64, 3, strides=1, padding='same'),
    BatchNormalization(),
    LeakyReLU(),

    Conv2DTranspose(64, 3, strides=2, padding='same'),
    BatchNormalization(),
    LeakyReLU(),

    Conv2DTranspose(64, 3, strides=2, padding='same'),
    BatchNormalization(),
    LeakyReLU(),

    Conv2DTranspose(32, 3, strides=1, padding='same'),
    BatchNormalization(),
    LeakyReLU(),

    Conv2DTranspose(1, 3, strides=1, padding='same', activation='sigmoid')
])

decoder.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 3136)              53312     
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
batch_normalization_4 (Batch (None, 7, 7, 64)          256       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 64)        36928     
_________________________________________________________________
batch_normalization_5 (Batch (None, 14, 14, 64)       

## Encoder + Decoder 합쳐서 학습

In [ ]:
encoder_in = Input(shape=(28, 28, 1))
x = encoder(encoder_in)
decoder_out = decoder(x)


auto_encoder = Model(encoder_in, decoder_out)

auto_encoder.compile(
    loss='mse',
    optimizer='sgd'
)

auto_encoder.fit(x_train, x_train, epochs=15)

# 1. Timeseries anomaly detection using an Autoencoder
## - conv1d사용

https://keras.io/examples/timeseries/timeseries_anomaly_detection/

input shape, output shape = (batch_size, timestep, num_features)

-conv1d는 tf version 2.3에서 가능

In [13]:
model = Sequential(
    [
        layers.Input(shape=(90, 79)),
        layers.Conv1D(
            filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1D(
            filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        layers.Conv1DTranspose(
            filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1DTranspose(
            filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        tf.kears.layers.Conv1DTranspose(filters=79, kernel_size=7, padding="same"),
    ]
)
model.compile(optimizer='adam', loss="mse")
model.summary()

AttributeError: module 'keras.layers' has no attribute 'Conv1DTranspose'

In [ ]:
history = model.fit(x_train,
                    x_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.1)

# 2. Time Series Anomaly Detection with LSTM Autoencoders using Keras in Python

https://www.curiousily.com/posts/anomaly-detection-in-time-series-with-lstms-using-keras-in-python/

https://towardsdatascience.com/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352

# 층 요약
encoder: LSTM_1(128, return_sequences=True)
         LSTM_2(64, return_sequences=False)
         
RepeatVector_layer

decoder: LSTM_3(64, return_sequences=True)
         LSTM_4(128, return_sequences=False)
         TimeDistributed(Dense(79))


## Encoder
#### return_sequences=True: LSTM layer가 sequence를 만듬.

return_suquences = True 이면 LSTM에서 매 cell이 output을 내놓음.
원래 input = (batchsize, timestep, features) 이면

True인 LSTM   -> output = (batchsize, timestep, cell갯수) 
False인 LSTM  -> output = (batchsize, cell갯수)

LSTM_2를 지나면 (batchsize, cell 갯수)로 encoding 됨

##  RepeatVector layer: 단순히 input을 n번 반복함

RepeatVector layer를 통해, encoding된 정보가 단순히 n개 복사되어서
output = (batchsize, n, cell)로 늘어남. 
encoder와 decoder를 이어주는 다리 역할 (만약에 다음 layer가 단순히 Dense 라면, 이 RepeatVector layer 필요없음)

## Decoder
#### Encoder의 역순

LSTM_3 -> output=(batchsize, n, cell)
LSTM_3 -> output=(batchsize, n, cell)

####  TimeDistributed layer: 전 layer의 outputs의 숫자와 같은 길이의 벡터를 생성.
전의 output이 (n,cell)인데 여기서 (cell, 1)의 벡터를 생성하고 79번 곱한다.
그러면 output이 맨처음 input이랑 형태가 같아진다.


In [6]:
ts = 90
model3 = Sequential()

model3.add(layers.LSTM(128, activation='relu',
                input_shape=(ts,89), return_sequences=True))
model3.add(layers.LSTM(64, activation='relu', return_sequences=False))
## encoder
model3.add(layers.RepeatVector(ts))
model3.add(layers.LSTM(64, activation='relu', return_sequences=True))
model3.add(layers.LSTM(128, activation='relu', return_sequences=True))
model3.add(layers.TimeDistributed(layers.Dense(89)))


model3.compile(optimizer='adam', loss='mse')
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 90, 128)           111616    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 90, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 90, 64)            33024     
_________________________________________________________________
lstm_3 (LSTM)                (None, 90, 128)           98816     
_________________________________________________________________
time_distributed (TimeDistri (None, 90, 89)            11481     
Total params: 304,345
Trainable params: 304,345
Non-trainable params: 0
__________________________________________________